In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from __init__ import *
import numpy as np
from svg_reader import SVGReader
from helpers.helpers import filter_none
from placement.attract import *
from visuals.plotter import plot_general
from copy import deepcopy
import networkx as nx
from itertools import pairwise, accumulate, product
from functools import reduce
from operator import add
from helpers.helpers import sort_and_group_objects
from domains.domain import Domain
from placement.neighbors import *
from placement.cardinal import *
from identify.id_problems import report_problems
from actions.selection import *
from helpers.layout import Layout
import plotly.io as pio
from icecream import ic


In [4]:

from pathlib import Path
svgs = Path.cwd().parent.parent / "svg_imports"

In [5]:
case  = svgs / f"red_d3.svg"
sv = SVGReader(case)
sv.run()
domains = sv.domains
pos = create_pos(domains)


In [6]:
[(i.name, i.area) for i in sv.init_domains.values()]

[('kitchen', Decimal('2.7176')),
 ('living', Decimal('5.1192')),
 ('transit', Decimal('2.1816')),
 ('bedroom_2', Decimal('2.2350')),
 ('bedroom', Decimal('2.7702')),
 ('closet', Decimal('0.4284')),
 ('transit_2', Decimal('0.5934')),
 ('bath', Decimal('1.3680'))]

In [7]:
plot_general(domains)

In [8]:
ad_layout = adjust_domains(domains)
a_domains, [Gx, Gy] = ad_layout
Gxc, Gyc = create_cardinal_dags(ad_layout)


In [9]:
plot_general(ad_layout.domains)

In [10]:
fl = FixLayout(ad_layout)
fl.run_to_completion()

initializing.. 0
next best layout closet-STRETCH-for-HOLE near ['closet', 'bedroom']1-PS:0.06-S:0.53
running again 1
next best layout transit-PULL-for-HOLE near ['transit', 'bedroom']1-PS:0.08-S:0.54
running again 2
next best layout bath-PUSH-for-OVERLAP near ['transit', 'bath']0-PS:0.04-S:0.02
No more results!
Found best layout: bath-PUSH-for-OVERLAP near ['transit', 'bath']0-PS:0.04-S:0.02


In [11]:
from actions.leveler import level_sides


new_doms = level_sides(fl.bl.layout)

In [12]:
plot_general(new_doms)